In [ ]:
%pylab inline

In [ ]:
num_steps=7000 # We know (empirically) 7000 steps to go from minimum to maximum

x=np.arange(num_steps)

print(3000000/num_steps) # Average inter-step interval

desired_y=(49.4145 / num_steps * (x+1) ) 

In [ ]:
# The measured movement follows this quadratic behavior. (Empirical, fit to a quadratic)
a=0.0071
b=0.7059
c=-0.3251
xr=np.linspace(-35,35,num_steps)
real_pos=a*xr**2+b*xr+c

In [ ]:
# Not used, but can map from non-linear to linear position
# xr_posh
# desired_pos=(-b+np.sqrt(b**2-4*a*(-desired_y)))/(2*a)
# np.interp(3500,desired_y,desired_pos)
# plt.plot( desired_pos, desired_y)

In [ ]:
# Solve to get a mapping as a function of linear variable (here time)
x_time=np.linspace(0,3.415,num_steps) # Not sure why 3.415
a,b,c=np.polyfit(x_time,real_pos,2)
print( a, b, c)
c_orig=c

In [ ]:
xr_steps=np.arange(7000)
fit_quad_pos=np.polyfit(xr_steps,real_pos,2)
print( fit_quad_pos )


In [ ]:
# So, invert to find the time between steps (the xs) that yield equally spaced desired_y's
a=2.983135 #3.8687
b=4.28199 #4.8654
c=0.4721
desired_times=(-b+np.sqrt(b**2-4*a*(-desired_y)))/(2*a)
intervals=np.diff(desired_times)

In [ ]:
plt.figure(figsize=(15,5))
offset=c_orig
plt.plot( x_time, a*x_time**2+ b*x_time+c*0+c_orig-offset)
plt.plot( x_time, real_pos-offset, '-', lw=3 )

plt.xlabel("Time (sec)", size=18)
plt.ylabel("Physical position\n (approx)", size=18)
plt.grid()
#plt.plot( x_time, a*x_time**2+ b*x_time+-16 )

# This isn't right:
if False:
    plt.plot(xr_steps/7000*x_time[-1], fit_quad_pos[0]*xr_steps**2+fit_quad_pos[1]*xr_steps+fit_quad_pos[0], 'g:')
    xl1=plt.xlim(0,x_time[-1])
    ax2=plt.twiny()

    xt=linspace(0,7000,8)

    ax2.set_xticks(xt/7000);
    #xt=np.linspace(0,)
    ax2.set_xticklabels(xt);
    ax2.set_xlabel("Step #", size=18)

In [ ]:
plt.figure(figsize=(15,4))

plt.grid()

In [ ]:
np.interp(desired_times[-1]/2, desired_times, xr_steps)
# Excellent, Very close to Chloe's empirical estimate of -32.5+10 to 32.5+10 !

In [ ]:
desired_y[0:5],desired_y[-5:]

In [ ]:
intervals[0:],intervals[-5:] # These match Chloe's spreadsheet

In [ ]:
intervals_usec = intervals * 1000000 # Convert to usec

In [ ]:
# Time between steps. This gets shorter as time goes on, since the motor needs
# to "speed up" to compensave for the nonlinear behavior of the vertical movement
# of the scissors lift (compared to the linear horizontal motion).
plt.plot(x[1:],intervals_usec)
plt.ylabel("Delay btwn steps (usec)", size=18)
plt.xlabel("Step #", size=18)
plt.grid()

#plt.semilogx()
ax2=plt.twinx()

In [ ]:
# Scale the intervals to uint8 for the table, going from 0-254 (linear)
# We lose precision, of course, but hopefully with rounding are
# still close to the desired behavior.

INTERVAL_TABLE_MAX=254.0 # uint8, from 0-254

expander_exponent=11 # To get a more accurate int, multiply/divide by this (2k)

interval_min=np.round( np.min(intervals_usec) )
interval_max=np.max(intervals_usec)
scaler=((interval_max-interval_min)/INTERVAL_TABLE_MAX)
scaler_expanded=np.round( scaler*(2**expander_exponent) )
if scaler==0: scaler=1 # For constants scalar might be 0

intervals_int8=np.round( (intervals_usec-interval_min)/scaler )
intervals_int8_recon = np.array(
    intervals_int8 * scaler_expanded/(2**expander_exponent) + interval_min, dtype=int)

In [ ]:
interval_min,interval_max,scaler,scaler_expanded,intervals_int8

In [ ]:
np.sum(intervals_int8_recon[250:-250]) # Should be close to 3sec

In [ ]:
intervals_int8[0:5],intervals_int8[-5:]

In [ ]:
fil=open('lookup_table1.h','wt')
fil.writelines('const unsigned int table1_info[] = {\n')
fil.writelines('\t%d, //num_steps\n'%num_steps);
fil.writelines('\t%d, //const unsigned int table_scaler = ;\n'%scaler_expanded);
fil.writelines('\t%d, //const unsigned int table_expander_exponent = ;\n'%expander_exponent);
fil.writelines('\t%d}; //const unsigned int table_interval_min = };\n'%interval_min);
fil.writelines('const PROGMEM uint8_t table1[] = {\n');
for nelement in intervals_int8:
    fil.writelines('%d,\n'%nelement)
fil.writelines('};\n');
fil.close()

In [ ]:
if np.max(intervals_int8)>2**8:
    print("Problem, exceeded 8-bit table.")
else:
    print("Ok. Please close/reopen Arduino IDE and recompile")
    print( datetime.datetime.now() )

In [ ]:
%load_ext autoreload
%autoreload 2

import luts

In [ ]:
# I don't understand why this plot doesn't go all the way up to 42.5 (_END)
plt.figure(figsize=(15,3))
offset=-(-22.5-c_orig)
plt.plot( x_time, a*x_time**2+ b*x_time+c_orig-offset)
plt.grid()


In [ ]:
luts.STEPPER1_MAX_FUDGE * 1.020, 6750/7000, 7000 - 7000 * luts.STEPPER1_MAX_FUDGE2

In [ ]:
luts.STEPPER1_END,luts.STEPPER1_START, luts.STEPPER1_END - luts.STEPPER1_START

In [ ]:
luts.get_pos(-1,1,3)
print()
luts.get_pos(1,-1.0,3)
print()
luts.get_pos(-1,0.5,3)
print()
print()
luts.get_pos(0,0.5,3)
print()
luts.get_pos(0,-0.5,3)
print()
print()
luts.get_pos(-0.5,0.5,3)
print()
luts.get_pos(0.5,-0.5,3)
print()
print()
luts.get_pos(-0.1,0.1,3)
print()
luts.get_pos(0.1,-0.1,3)

In [ ]:
luts.xr_steps[3000::-1]

In [ ]:
luts.get_pos(0,1,3)

In [ ]:
luts.get_pos(-1,2250,3)

In [ ]:
luts.get_pos(0,2250,3)

In [ ]:
luts.get_pos(1,-6500,3)

In [ ]:
luts.get_pos(1,-1,3)

In [ ]:
luts.get_pos(-0.1,0.1,3)

In [ ]:
luts.get_pos(0.1,-0.1,3)

In [ ]:
plt.plot( luts.desired_times)

In [ ]:
luts.intervals[6750:250:-1]